In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import pprint as pp

import requests
import json
from difflib import get_close_matches
from difflib import SequenceMatcher

from sklearn.metrics.pairwise import cosine_similarity

from scipy.spatial.distance import cosine

plt.style.use('seaborn')

In [2]:
genres_item_matrix = pd.read_csv('data/genres_item_matrix.csv', index_col='id')
tag_item_matrix = pd.read_csv('data/tag_item_matrix.csv', index_col='id')
tag_rank_item_matrix = pd.read_csv('data/tag_rank_item_matrix.csv', index_col='id')
title_df = pd.read_csv('data/title_df.csv', index_col='id')
adjusted_score_df = pd.read_csv('data/adjusted_score_df.csv', index_col='id')

In [3]:
binary_df = pd.concat([genres_item_matrix, tag_item_matrix], axis=1)
binary_df.head()

,action,adventure,comedy,drama,ecchi,fantasy,horror,mahou_shoujo,mecha,music,...,witch,work,wrestling,writing,wuxia,yakuza,yandere,youkai,yuri,zombie
id,,,,,,,,,,,,,,,,,,,,,
10161,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
99726,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98526,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
966,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4876,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
rank_df = pd.concat([genres_item_matrix, tag_rank_item_matrix], axis=1)
rank_df.head()

,action,adventure,comedy,drama,ecchi,fantasy,horror,mahou_shoujo,mecha,music,...,witch,work,wrestling,writing,wuxia,yakuza,yandere,youkai,yuri,zombie
id,,,,,,,,,,,,,,,,,,,,,
10161,1,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.44,0.0,0.0,0.0
99726,0,1,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
98526,1,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
966,0,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4876,1,1,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [5]:
sim_mat = cosine_similarity(rank_df.values)

In [6]:
a = .1
adjusted_scores = ((adjusted_score_df['adjusted_score'] * a) + .9)
adjusted_scores

id
10161     0.965029
99726     0.969294
98526     0.952263
966       0.965896
4876      0.954932
            ...   
99586     0.959066
99916     0.968809
101283    0.961290
101633    0.966854
101089    0.900000
Name: adjusted_score, Length: 9832, dtype: float64

In [7]:
weighted_sim_mat = (sim_mat * adjusted_scores.to_numpy())
weighted_sim_mat

array([[0.96502869, 0.        , 0.47993428, ..., 0.        , 0.14642307,
        0.        ],
       [0.        , 0.96929419, 0.        , ..., 0.05666303, 0.22477452,
        0.32280893],
       [0.48636803, 0.        , 0.95226315, ..., 0.        , 0.21052595,
        0.        ],
       ...,
       [0.        , 0.05713483, 0.        , ..., 0.96129011, 0.        ,
        0.03971541],
       [0.14614666, 0.22534184, 0.20734891, ..., 0.        , 0.96685389,
        0.31327804],
       [0.        , 0.34766313, 0.        , ..., 0.04242003, 0.33654899,
        0.9       ]])

In [8]:
def get_anime_id(_user_preferred_title):
    id_list = []
    anime_id = title_df[title_df['userPreferred'].isin([_user_preferred_title])].index
    id_list.append(anime_id)
    
    return id_list[0].values[0]


def get_top_n_recommendations(anime_id, dataframe, similarity_matrix, n=5):
    positional_idx = dataframe.index.get_loc(anime_id)
    
    top_n = np.argsort(similarity_matrix[positional_idx,:])[-n-1:-1]
    recom_titles = []
    for idx, row in title_df.iloc[top_n,:].iterrows():
        if type(row['english']) != float:
            recom_titles.append(row['english'])
        else:
            recom_titles.append(row['userPreferred'])
    
    return recom_titles


        
def get_title_from_id(_id):
    '''Searches the title dataframe based on an anime id and tries to return the english title. 
    If an english title is not available, the "user preferred" is give. '''
    title = None
    if pd.isna(title_df.loc[title_df.index == _id, 'english']).values[0]:
        title = title_df.loc[title_df.index == _id, 'userPreferred'].values[0]
    else:
        title = title_df.loc[title_df.index == _id, 'english'].values[0]
    
    return title


def get_user_preferred(initial_search):
    '''Search the AniList API for a show based on a query.
    
    Returns the user preferred title 
    '''
    query = '''
    query ($search: String) {
      Media (type: ANIME, search: $search) {
        id
        title {
          romaji
          english
          native
          userPreferred
        }
      }
    }
    '''
    variables = {'search': initial_search}

    url = 'https://graphql.anilist.co'
    response = requests.post(url, 
                             json={'query': query, 
                                   'variables': variables})
    user_preferred_title = response.json()['data']['Media']['title']['userPreferred']

    return user_preferred_title


def view_features(search_term, df):
    _id = title_df[title_df['userPreferred'] == search_term].index
    _df = df.loc[_id,:]

    return list(_df.loc[:, (_df != 0).any(axis=0)].columns)



def view_features_from_id(_id, _df):
    single_show = _df.loc[_id,:]
    
    return set(single_show[single_show != 0].index)



def get_common_tags(list_of_ids):
    list_of_tags = [view_features_from_id(_id, binary_df) for _id in list_of_ids]

    return set.intersection(*list_of_tags)

In [9]:
def get_recommendations(search_term, similarity_matrix=weighted_sim_mat):
    user_preferred_title = get_user_preferred(search_term)
    _id = get_anime_id(user_preferred_title)
    print('Getting recommendations for: ', get_title_from_id(_id))
    rec_list = get_top_n_recommendations(_id, 
                                         rank_df, 
                                         similarity_matrix, 
                                         n=5)
    return rec_list



In [10]:
# list of titles to list of ids
title_list = ['Attack on Titan', 
              'Black Lagoon', 
              'Blood Blockade Battlefront', 
              'Cowboy Bebop', 
              'FLCL', 
              'Fullmetal Alchemist: Brotherhood', 
              'Shokugeki no Soma', 
              'One-Punch Man']
id_list = []
for title in title_list:
    user_preferred_title = get_user_preferred(title)
    id_list.append(get_anime_id(user_preferred_title))
    print(user_preferred_title)

Shingeki no Kyojin
Black Lagoon
Kekkai Sensen
Cowboy Bebop
FLCL
Hagane no Renkinjutsushi: Fullmetal Alchemist
Shokugeki no Souma
One Punch Man


In [11]:
rank_df.loc[id_list,:]

,action,adventure,comedy,drama,ecchi,fantasy,horror,mahou_shoujo,mecha,music,...,witch,work,wrestling,writing,wuxia,yakuza,yandere,youkai,yuri,zombie
id,,,,,,,,,,,,,,,,,,,,,
16498,1,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
889,1,1,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
20727,1,1,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
1,1,1,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.47,0.0,0.0,0.0,0.0
227,1,0,1,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
5114,1,1,1,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
20923,1,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
21087,1,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [12]:
# get the mean vector for a list of anime
mean_vector = rank_df.loc[id_list,:].mean(axis=0)
mean_vector

action       1.00000
adventure    0.50000
comedy       0.75000
drama        0.50000
ecchi        0.12500
              ...   
yakuza       0.05875
yandere      0.00000
youkai       0.00000
yuri         0.00000
zombie       0.00000
Length: 265, dtype: float64

In [13]:
# measure the cosine similarity between our mean vector and every item in our dataset
dist_list = []
for anime in rank_df.iterrows():
    dist_list.append(cosine(mean_vector, anime[1]))
dist_series = pd.Series(dist_list, index=rank_df.index)

/home/sean/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [15]:
dist_series

id
10161     0.620737
99726     0.687303
98526     0.558353
966       0.679230
4876      0.400080
            ...   
99586     0.950964
99916     0.965916
101283    0.849458
101633    0.772254
101089    0.990800
Length: 9832, dtype: float64

In [16]:
title_list

['Attack on Titan',
 'Black Lagoon',
 'Blood Blockade Battlefront',
 'Cowboy Bebop',
 'FLCL',
 'Fullmetal Alchemist: Brotherhood',
 'Shokugeki no Soma',
 'One-Punch Man']

In [17]:
# USING AVERAGE FEATURE VECTOR TO FIND MOST SIMILAR ELEMENTS IN A SIMILARITY MATRIX
# get the titles for our ids
for num, _id in enumerate(list(dist_series.sort_values()[:30].index)):
    print(num+1, get_title_from_id(_id))

1 Trigun
2 Dragon Ball Movie 1: Curse of the Blood Rubies
3 Dragon Ball Movie 2: Sleeping Princess in Devil's Castle
4 Cowboy Bebop
5 Dragon Ball Z: The Tree of Might
6 Dragon Ball Z: Bio-Broly
7 One Piece Film: Gold
8 Dragon Ball Z: Lord Slug
9 Dragon Ball Z: Broly - The Legendary Super Saiyan
10 Fullmetal Alchemist
11 Fullmetal Alchemist: Brotherhood
12 Dragon Ball Movie 4: The Path to Power
13 Dragon Ball Z: Wrath of the Dragon
14 One Piece: Heart of Gold
15 Dragon Ball Z: Bojack Unbound
16 Dragon Ball Z: Cooler's Revenge
17 Dragon Ball Z: Super Android 13!
18 Dragon Ball Z: Broly - Second Coming
19 Dragon Ball Movie 3: Mystical Adventure
20 Blood Blockade Battlefront: King of Kings Restaurant Fit for a King
21 Blood Blockade Battlefront
22 Dragon Ball: Episode of Bardock
23 One Piece Film: Strong World
24 Dragon Ball Z Movie: Return of Cooler
25 R.O.D -THE TV-
26 Toriko: Jump Super Anime Tour 2009 Special
27 One Piece - Episode of Sabo: Bond of Three Brothers - A Miraculous Reunion